In [27]:
import oss2
import json
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import requests
with open("../aliyun_key.json" ,'r') as key:
    AccessKeyDict=json.loads(key.read())
AccessKeyId=AccessKeyDict["AccessKeyId"]
AccessKeySecret=AccessKeyDict["AccessKeySecret"]
auth = oss2.Auth(AccessKeyId, AccessKeySecret)
bucket = oss2.Bucket(auth, 'https://oss-cn-shanghai.aliyuncs.com', 'ocr-yjs')
from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdkcore.auth.credentials import AccessKeyCredential
from aliyunsdkcore.auth.credentials import StsTokenCredential
credentials = AccessKeyCredential(AccessKeyId, AccessKeySecret)
client = AcsClient(region_id='cn-shanghai', credential=credentials)
import seaborn as sns
import uuid
from urllib.parse import quote

In [17]:
"""
https://help.aliyun.com/document_detail/203512.html
添加人脸图像模板
"""
from aliyunsdkfacebody.request.v20191230.AddFaceImageTemplateRequest import AddFaceImageTemplateRequest

request = AddFaceImageTemplateRequest()
request.set_accept_format('json')

filepath="images/test2.jpg"


def get_url(filepath):
    with open(filepath, 'rb') as fileobj:
        bucket.put_object(filepath, fileobj)
    url="https://ocr-yjs.oss-cn-shanghai.aliyuncs.com/"+filepath
    return url

url=get_url(filepath)   
UserId=str(uuid.uuid4())
request.set_UserId(UserId)

request.set_ImageURL(url)

response = client.do_action_with_exception(request)
response=json.loads(str(response, encoding='utf-8'))
print(response)

TemplateId=response['Data']['TemplateId']

{'RequestId': '66DEABAE-C9E9-5B71-82B5-75AFFDFFBFBD', 'Data': {'TemplateId': '839a8073-2a11-4ef2-99e0-48f1ee626f71'}}


In [18]:
"""
https://help.aliyun.com/document_detail/203511.html
图像人脸融合模板查询
图像人脸融合模板查询功能可以查询已创建的人脸模板。
"""
from aliyunsdkfacebody.request.v20191230.QueryFaceImageTemplateRequest import QueryFaceImageTemplateRequest
try:
    request = QueryFaceImageTemplateRequest()
    request.set_accept_format('json')

    request.set_UserId(UserId)
    request.set_TemplateId(TemplateId)


    response = client.do_action_with_exception(request)
    response=json.loads(str(response, encoding='utf-8'))
    print(response)
except Exception as error:
    print(error)


{'RequestId': 'E7CD1102-4753-5243-9263-C7E1C04D6A3B', 'Data': {'Elements': [{'TemplateURL': 'http://vibktprfx-prod-prod-aic-gd-cn-shanghai.oss-cn-shanghai.aliyuncs.com/image-face-fusion/OriginUrl/user/2021-08-21/cc769a7a-afc7-4571-b919-ad5e65a45546?Expires=1629512823&OSSAccessKeyId=LTAI4FoLmvQ9urWXgSRpDvh1&Signature=WmlcXjSjksXdO0QJZ3hFTGVFuX8%3D', 'UserId': '88148e52-bc8f-4d59-b60e-e8f4ea01a16d', 'CreateTime': '2021-08-21 09:57:00', 'UpdateTime': '2021-08-21 09:57:00', 'TemplateId': '839a8073-2a11-4ef2-99e0-48f1ee626f71'}]}}


In [7]:
"""
https://help.aliyun.com/document_detail/203513.html
图像人脸融合模板删除
图像人脸融合模板查询功能可以删除已创建的人脸模板。
"""

from aliyunsdkfacebody.request.v20191230.DeleteFaceImageTemplateRequest import DeleteFaceImageTemplateRequest


request = DeleteFaceImageTemplateRequest()
request.set_accept_format('json')

request.set_UserId(UserId)
request.set_TemplateId(TemplateId)

response = client.do_action_with_exception(request)
response=json.loads(str(response, encoding='utf-8'))
print(response)

{'RequestId': 'D302BA51-8833-5716-A635-EFE67971F051'}


In [37]:

"""
https://help.aliyun.com/document_detail/203510.html
图像人脸融合
图像人脸融合功能可以帮助用户在获得授权的前提下，将图像中的指定的某个人脸，融合进另一人的人脸特征。
"""
from aliyunsdkfacebody.request.v20191230.MergeImageFaceRequest import MergeImageFaceRequest



request = MergeImageFaceRequest()
request.set_accept_format('json')


filepath="images/qqq.jpg"


def get_url(filepath):
    with open(filepath, 'rb') as fileobj:
        bucket.put_object(filepath, fileobj)
    url="https://ocr-yjs.oss-cn-shanghai.aliyuncs.com/"+filepath
    return url

url=get_url(filepath)
print(url)
request.set_UserId(UserId)
request.set_TemplateId(TemplateId)
request.set_ImageURL(url)

response = client.do_action_with_exception(request)
response=json.loads(str(response, encoding='utf-8'))
print(response)

https://ocr-yjs.oss-cn-shanghai.aliyuncs.com/images/qqq.jpg
{'RequestId': '6336B1FA-7F93-57DE-9078-D80C384DE6F1', 'Data': {'ImageURL': 'http://vibktprfx-prod-prod-aic-gd-cn-shanghai.oss-cn-shanghai.aliyuncs.com/image-face-fusion/b83ba80b-99a4-4448-8a9c-a29a752a40ab_7b04_20210821-021716.jpg?Expires=1629514036&OSSAccessKeyId=LTAI4FoLmvQ9urWXgSRpDvh1&Signature=Je3SjKEQAXfKmBqWBj580sL6CQw%3D'}}


In [38]:
ImageURL=response["Data"]["ImageURL"]

ImageURL

'http://vibktprfx-prod-prod-aic-gd-cn-shanghai.oss-cn-shanghai.aliyuncs.com/image-face-fusion/b83ba80b-99a4-4448-8a9c-a29a752a40ab_7b04_20210821-021716.jpg?Expires=1629514036&OSSAccessKeyId=LTAI4FoLmvQ9urWXgSRpDvh1&Signature=Je3SjKEQAXfKmBqWBj580sL6CQw%3D'

In [16]:
"""
UserId	String	是	zhangsan	
人脸模板的用户ID。

TemplateId	String	是	6cd509ea-54fa-4730-8e9d-c94cadcda048	
人脸模板的模板ID。

ImageURL	String	是	http://invi-label.oss-cn-shanghai.aliyuncs.com/label/temp/cartoon/test_data/g02.jpeg	
待融合的人脸照片。当前仅支持上海地域的OSS链接，如何生成URL请参见生成URL。
"""

'\nSharp\tFloat\t是\t1\t\n锐化（清晰度）程度，取值范围0~1。越接近1锐化程度越高。\n\nSmooth\tFloat\t是\t0.1\t\n平滑程度，取值范围0~1。越接近1平滑程度越高。\n\nWhite\tFloat\t是\t0.3\t\n美白程度，取值范围0~1。越接近1美白程度越高。\n'

In [39]:
with open("images/图像人脸融合后.jpg",'wb') as file:
    file.write(requests.get(ImageURL).content)